# Preprocessing data
In general, learning algorithms benefit from standardization of the data set. If some outliers are present in the set, robust scalers or transformers are more appropriate

# 4.1 Standardization, or mean removal and variance scaling
Standardization of datasets so that the individual features will have zero mean and unit variance

### Default scale methods parameters
`sklearn.preprocessing.scale(X, axis=0, with_mean=True, with_std=True, copy=True)`

Documentation: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html#sklearn.preprocessing.scale

In [1]:
import numpy as np
from sklearn import preprocessing
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

In [2]:
X_scaled = preprocessing.scale(X=X_train, axis=0, with_mean=True, with_std=True, copy=True)
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [3]:
X_scaled.mean(axis=0)

array([ 0.,  0.,  0.])

In [4]:
X_scaled.std(axis=0)

array([ 1.,  1.,  1.])

### How axis works
`axis=0` means standardize each feature/column
`axis=1` means standardize each sample/row

In [5]:
X_scaled = preprocessing.scale(X=X_train, axis=1, with_mean=True, with_std=True, copy=True)
X_scaled

array([[ 0.26726124, -1.33630621,  1.06904497],
       [ 1.41421356, -0.70710678, -0.70710678],
       [ 0.        ,  1.22474487, -1.22474487]])